<a href="https://colab.research.google.com/github/yukinaga/ai_stock_prediction/blob/main/section_3/01_nn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIによるシンプルな株価予測（データの取得・学習・推論）
ここでは、yfinanceでデータを取得し、前日比（変動率）に変換した後、過去30日間の変動率から翌日の変動率を予測するシンプルなモデルを学習させます。

In [ ]:
# 必要なライブラリのインストールとインポート
# !pip install -q yfinance

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# --- 設定 ---
STOCK_CODE = '7203.T'  # 予測対象（例: トヨタ自動車）
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
WINDOW_SIZE = 15     # 過去何日分のデータを見るか
TEST_RATIO = 0.2     # テストデータの割合
EPOCHS = 50
BATCH_SIZE = 32

# 1. データの取得と前処理
print("データを取得中...")
df = yf.download(STOCK_CODE, start=START_DATE, end=END_DATE, progress=False)

# 株価そのものではなく「変動率（リターン）」を計算
# pct_change()で (今日の価格 - 前日の価格) / 前日の価格 を計算
data = df['Close'].pct_change().dropna().values.reshape(-1, 1)

# データの正規化（学習を安定させるため）
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)

# データセットの作成関数（スライディングウィンドウ）
def create_dataset(dataset, window_size):
    X, y = [], []
    for i in range(len(dataset) - window_size):
        feature = dataset[i:i + window_size]
        target = dataset[i + window_size]
        X.append(feature)
        y.append(target)
    return np.array(X), np.array(y)

X, y = create_dataset(data_normalized, WINDOW_SIZE)

# 学習用とテスト用に分割
split_index = int(len(X) * (1 - TEST_RATIO))
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

# PyTorchのTensorに変換
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

# データローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 2. シンプルなニューラルネットワークの定義
# RNNではなく、全結合層(Linear)のみのシンプルな構成
class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1) # 出力は1つ（翌日の変動率）

    def forward(self, x):
        # 入力形状: (Batch, Window, 1) -> (Batch, Window) にフラット化
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet(WINDOW_SIZE).to(device)

# 損失関数と最適化手法
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. 学習ループ
print("学習を開始します...")
train_losses = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.6f}')

print("学習完了。")

# 学習曲線の表示
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

# 予測結果の評価 その1（視覚化と誤差の計算）
ここでは、学習データには含まれないテストデータを使って推論を行い、実際にどれくらい「変動率（上がり下がり）」を当てられたかを確認します。

In [ ]:
# 評価モードへ
model.eval()

# 推論の実行
with torch.no_grad():
    predictions = model(X_test_tensor.to(device)).cpu().numpy()

# 正規化を元に戻す（実際の変動率%に戻す）
predictions_actual = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# グラフによる可視化（全期間だと見づらいため、直近100日分を表示）
plt.figure(figsize=(14, 6))
# 実際の変動率
plt.plot(y_test_actual[-100:], label='Actual Return', color='blue', alpha=0.6)
# 予測された変動率
plt.plot(predictions_actual[-100:], label='Predicted Return', color='red', alpha=0.7)

plt.title(f'{STOCK_CODE} Return Prediction (Last 100 Days)')
plt.xlabel('Time (Days)')
plt.ylabel('Return Rate (Fluctuation)')
plt.legend()
plt.grid(True)
plt.show()

# 誤差の計算 (RMSE)
rmse = np.sqrt(np.mean((predictions_actual - y_test_actual)**2))
print(f"Test RMSE (Root Mean Squared Error): {rmse:.6f}")
print("※ 変動率は0に近い非常に小さな値のため、RMSEも小さくなります。")

# 予測結果の評価 その2（投資シミュレーション）
予測精度（RMSE）だけでは実用性が分かりにくいため、投資シミュレーションを行います。 **「AIがプラス（株価が上がる）と予測したら買い、マイナス（下がる）と予測したら買わない（現金で保持）」**というシンプルな戦略の効果を検証します。

In [ ]:
# 投資シミュレーション
initial_capital = 100000.0  # 初期資金（例：10万ドル/円）

# 戦略1: AIトレード（予測 > 0 なら保有、予測 < 0 なら現金）
ai_capital = [initial_capital]
current_ai_capital = initial_capital

# 戦略2: ガチホ（Buy and Hold: 最初に買ってずっと持ち続ける）
hold_capital = [initial_capital]
current_hold_capital = initial_capital

for i in range(len(y_test_actual)):
    actual_return = y_test_actual[i][0]   # その日の実際の変動率
    predicted_return = predictions_actual[i][0] # AIが予測した変動率

    # --- AI戦略 ---
    # AIが「上がる」と予測した場合のみ、その日の変動を受け入れる
    if predicted_return > 0:
        current_ai_capital = current_ai_capital * (1 + actual_return)
    else:
        # 「下がる」と予測したらポジションを持たないので資産は変わらない（手数料は無視）
        pass
    ai_capital.append(current_ai_capital)

    # --- ガチホ戦略 ---
    # 常に変動を受け入れる
    current_hold_capital = current_hold_capital * (1 + actual_return)
    hold_capital.append(current_hold_capital)

# 結果の可視化
plt.figure(figsize=(14, 6))
plt.plot(ai_capital, label='AI Strategy (Simple NN)', color='red')
plt.plot(hold_capital, label='Buy & Hold Strategy', color='blue', linestyle='--')

plt.title('Investment Simulation: AI vs Buy & Hold')
plt.xlabel('Days')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()

print(f"最終資産 (Buy & Hold): {hold_capital[-1]:.2f}")
print(f"最終資産 (AI Strategy): {ai_capital[-1]:.2f}")

if ai_capital[-1] > hold_capital[-1]:
    print("結果: AI戦略が市場平均を上回りました。")
else:
    print("結果: AI戦略は市場平均に及びませんでした（予測の難しさを示しています）。")